# Exploratory Data Analysis of Zillow Data Set

This data is from the Kaggle competition to improve Zillow's "Zestimate": https://www.kaggle.com/c/zillow-prize-1

The data:

* properties_2017.csv: a sample of all properties from 2017 listed on Zillow through Sept
* properties_2016.csv: a sample of all properties from 2016 listed on Zillow
* train_2017.csv: contains dates, propertyids, and logerror for each transaction in 2017 through Sept
* train_2016_v2.csv: contains dates, propertyids, and logerror for each transaction in 2016
* Not all properties have transactions
* logerror=log(Zestimate)−log(SalePrice)

Goal: find a model that reduces the logerror

In [11]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from matplotlib.pyplot import cm

In [12]:
pd.set_option('display.max_columns', 65)

# Data Ingestion

In [13]:
df16 = pd.read_csv('properties_2016.csv', low_memory=False)
df_transactions16 = pd.read_csv('train_2016_v2.csv', low_memory=False)
df_merged16 = pd.merge(df16, df_transactions16, on='parcelid', how='right')
df_merged16.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,548.0,1264.0,1264.0,NaN,NaN,548.0,NaN,6111.0,NaN,2.0,2.0,0.0,NaN,NaN,34303597.0,-119287236.0,1735.0,NaN,NaN,NaN,NaN,NaN,1128,265.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97081.0,5.0,NaN,1.0,NaN,NaN,128.0,NaN,1986.0,2.0,NaN,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,777.0,777.0,777.0,NaN,NaN,777.0,NaN,6111.0,NaN,1.0,1.0,0.0,NaN,NaN,34272866.0,-119198911.0,NaN,NaN,NaN,NaN,NaN,NaN,1129,266.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97083.0,4.0,NaN,NaN,NaN,NaN,198.0,NaN,1990.0,1.0,NaN,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,0.0198,2016-03-30
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,1101.0,1101.0,1101.0,NaN,NaN,1101.0,NaN,6111.0,NaN,2.0,2.0,441.0,NaN,NaN,34340801.0,-119079610.0,6569.0,NaN,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111001e+07,26965.0,2061.0,NaN,97113.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1956.0,1.0,NaN,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.0060,2016-05-27
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,1554.0,1554.0,1554.0,NaN,NaN,1554.0,NaN,6111.0,1.0,1.0,2.0,460.0,NaN,NaN,34354313.0,-119076405.0,7400.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,26965.0,2061.0,NaN,97113.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN,1965.0,1.0,NaN,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.0566,2016-06-07
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,1305.0,2415.0,2415.0,NaN,NaN,1305.0,NaN,6111.0,1.0,2.0,2.0,665.0,NaN,NaN,34266578.0,-119165392.0,6326.0,NaN,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111001e+07,34543.0,2061.0,NaN,97084.0,8.0,NaN,1.0,NaN,NaN,NaN,NaN,1984.0,2.0,NaN,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.0573,2016-08-08


In [14]:
df17 = pd.read_csv('properties_2017.csv', low_memory=False)
df_transactions17 = pd.read_csv('train_2017.csv', low_memory=False)
df_merged17 = pd.merge(df17, df_transactions17, on='parcelid', how='right')
df_merged17.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,17054981,NaN,NaN,NaN,5.0,4.0,NaN,NaN,5.0,NaN,1968.0,3760.0,3760.0,NaN,NaN,1968.0,NaN,6111.0,4.0,5.0,3.0,784.0,NaN,NaN,34449407.0,-119254052.0,42688.0,1.0,735.0,NaN,NaN,1.0,1110,261.0,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,1982.0,2.0,NaN,501928.0,872850.0,2016.0,370922.0,9673.46,NaN,NaN,6.111001e+13,-0.013099,2017-06-15
1,17055743,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,66.0,1444.0,1444.0,1444.0,NaN,NaN,1444.0,NaN,6111.0,NaN,2.0,1.0,0.0,NaN,NaN,34454169.0,-119237898.0,7108.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1951.0,1.0,NaN,130845.0,436157.0,2016.0,305312.0,5538.80,NaN,NaN,6.111001e+13,0.073985,2017-07-26
2,17068109,NaN,NaN,NaN,1.5,3.0,NaN,NaN,1.5,66.0,1698.0,1698.0,1698.0,NaN,NaN,1698.0,NaN,6111.0,1.0,1.0,0.0,0.0,NaN,NaN,34365693.0,-119448392.0,2588.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,34543.0,2061.0,NaN,97081.0,6.0,NaN,1.0,NaN,NaN,NaN,NaN,1979.0,1.0,NaN,193413.0,286606.0,2016.0,93193.0,2987.36,NaN,NaN,6.111001e+13,0.071886,2017-07-28
3,17073952,NaN,NaN,NaN,2.0,2.0,NaN,NaN,2.0,NaN,986.0,986.0,986.0,NaN,NaN,986.0,NaN,6111.0,1.0,2.0,1.0,0.0,NaN,NaN,34305600.0,-119284000.0,NaN,NaN,NaN,NaN,NaN,NaN,1129,266.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97081.0,4.0,NaN,NaN,NaN,NaN,55.0,NaN,1989.0,1.0,NaN,90357.0,258888.0,2016.0,168531.0,2706.24,NaN,NaN,6.111002e+13,0.305680,2017-06-02
4,17078502,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1170.0,1170.0,1170.0,NaN,NaN,1170.0,NaN,6111.0,1.0,1.0,2.0,621.0,NaN,NaN,34278012.0,-119257047.0,5643.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97083.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1948.0,1.0,NaN,148752.0,592930.0,2016.0,444178.0,6220.70,NaN,NaN,6.111002e+13,-0.073787,2017-07-07


## Data Cleaning for Exploration

Looking at the data above, a few columns need to be cleaned up before we can do exploratory data analysis.

First, the latitudes and longitudes are missing their decimal points:

In [15]:
df_merged17['latitude'] = df_merged17['latitude'] / 1000000
df_merged17['longitude'] = df_merged17['longitude'] / 1000000
df_merged16['latitude'] = df_merged16['latitude'] / 1000000
df_merged16['longitude'] = df_merged16['longitude'] / 1000000

Second, the tax delinquency years are listed as YY, with the first digit missing if it is a 0. Since some of the years are from the 1990s, we need to fix this so that they will sort in the correct order:

In [16]:
def convertyears(x):
    if x > 9 and x < 20:
        t = '20' + str(x)
        return float(t)
    elif x <= 9:
        t = '200' + str(x)
        return float(t)
    elif x > 20:
        t = '19' + str(x)
        return float(t)
    else:
        return np.nan
    


df_merged17['taxdelinquencyyear'] = df_merged17['taxdelinquencyyear'].map(lambda a: convertyears(a))
df_merged16['taxdelinquencyyear'] = df_merged16['taxdelinquencyyear'].map(lambda a: convertyears(a))

Let's also do a quick check of data types:

In [17]:
df_merged16.dtypes

parcelid                          int64
airconditioningtypeid           float64
architecturalstyletypeid        float64
basementsqft                    float64
bathroomcnt                     float64
bedroomcnt                      float64
buildingclasstypeid             float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
decktypeid                      float64
finishedfloor1squarefeet        float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
finishedsquarefeet13            float64
finishedsquarefeet15            float64
finishedsquarefeet50            float64
finishedsquarefeet6             float64
fips                            float64
fireplacecnt                    float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
hashottuborspa                   object
heatingorsystemtypeid           float64
latitude                        float64


The transaction dates will be more useful in a datetime format. Categorical columns also have the wrong types, but we will deal with those on a case by case basis.

In [18]:
format = '%Y-%m-%d'
df_merged16['transactiondate'] = df_merged16['transactiondate'].map(lambda a: datetime.datetime.strptime(a, format))
df_merged17['transactiondate'] = df_merged17['transactiondate'].map(lambda a: datetime.datetime.strptime(a, format))

The pool types have been separated into one-hot columns. We are going to combine them, calculating based on poolcnt and hashottuborspa.

In [20]:
df_merged16['poolcnt'].fillna(0, inplace=True)
df_merged16['hashottuborspa'].fillna(False, inplace=True)
pools16 = pd.DataFrame(columns=['parcelid','pooltype'])     
for i, row in df_merged16.iterrows():
    if row['hashottuborspa'] and row['poolcnt'] > 0:
        pools16.loc[len(pools16)] = [row['parcelid'],2] 
    elif not(row['hashottuborspa']) and row['poolcnt'] > 0:
        pools16.loc[len(pools16)] = [row['parcelid'],7] 
    elif row['hashottuborspa'] and row['poolcnt'] == 0:
        pools16.loc[len(pools16)] = [row['parcelid'],10] 
    else:
        pools16.loc[len(pools16)] = [row['parcelid'],0] 
df_16p = pd.merge(df_merged16, pools16, on='parcelid', how='left')

df_merged17['poolcnt'].fillna(0, inplace=True)
df_merged17['hashottuborspa'].fillna(False, inplace=True)
pools17 = pd.DataFrame(columns=['parcelid','pooltype'])     
for i, row in df_merged17.iterrows():
    if row['hashottuborspa'] and row['poolcnt'] > 0:
        pools17.loc[len(pools17)] = [row['parcelid'],2] 
    elif not(row['hashottuborspa']) and row['poolcnt'] > 0:
        pools17.loc[len(pools17)] = [row['parcelid'],7] 
    elif row['hashottuborspa'] and row['poolcnt'] == 0:
        pools17.loc[len(pools17)] = [row['parcelid'],10] 
    else:
        pools17.loc[len(pools17)] = [row['parcelid'],0] 
df_17p = pd.merge(df_merged17, pools17, on='parcelid', how='left')

In some analyses we will be looking at the combined data sets:

In [68]:
df_16p['setyear'] = 2016
df_17p['setyear'] = 2017
df_total = df_16p.append(df_17p, ignore_index=True)
df_total.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate,pooltype,setyear
0,17073783,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,548.0,1264.0,1264.0,NaN,NaN,548.0,NaN,6111.0,NaN,2.0,2.0,0.0,False,NaN,34.303597,-119.287236,1735.0,0.0,NaN,NaN,NaN,NaN,1128,265.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97081.0,5.0,NaN,1.0,NaN,NaN,128.0,NaN,1986.0,2.0,NaN,115087.0,191811.0,2015.0,76724.0,2015.06,NaN,NaN,6.111002e+13,0.0953,2016-01-27,0,2016
1,17088994,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,777.0,777.0,777.0,NaN,NaN,777.0,NaN,6111.0,NaN,1.0,1.0,0.0,False,NaN,34.272866,-119.198911,NaN,0.0,NaN,NaN,NaN,NaN,1129,266.0,NaN,6.111002e+07,34543.0,2061.0,NaN,97083.0,4.0,NaN,NaN,NaN,NaN,198.0,NaN,1990.0,1.0,NaN,143809.0,239679.0,2015.0,95870.0,2581.30,NaN,NaN,6.111002e+13,0.0198,2016-03-30,0,2016
2,17100444,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,1101.0,1101.0,1101.0,NaN,NaN,1101.0,NaN,6111.0,NaN,2.0,2.0,441.0,False,NaN,34.340801,-119.079610,6569.0,0.0,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111001e+07,26965.0,2061.0,NaN,97113.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1956.0,1.0,NaN,33619.0,47853.0,2015.0,14234.0,591.64,NaN,NaN,6.111001e+13,0.0060,2016-05-27,0,2016
3,17102429,NaN,NaN,NaN,1.5,2.0,NaN,NaN,1.5,NaN,1554.0,1554.0,1554.0,NaN,NaN,1554.0,NaN,6111.0,1.0,1.0,2.0,460.0,False,NaN,34.354313,-119.076405,7400.0,0.0,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,26965.0,2061.0,NaN,97113.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN,1965.0,1.0,NaN,45609.0,62914.0,2015.0,17305.0,682.78,NaN,NaN,6.111001e+13,-0.0566,2016-06-07,0,2016
4,17109604,NaN,NaN,NaN,2.5,4.0,NaN,NaN,2.5,NaN,1305.0,2415.0,2415.0,NaN,NaN,1305.0,NaN,6111.0,1.0,2.0,2.0,665.0,False,NaN,34.266578,-119.165392,6326.0,0.0,NaN,NaN,NaN,NaN,1111,261.0,NaN,6.111001e+07,34543.0,2061.0,NaN,97084.0,8.0,NaN,1.0,NaN,NaN,NaN,NaN,1984.0,2.0,NaN,277000.0,554000.0,2015.0,277000.0,5886.92,NaN,NaN,6.111001e+13,0.0573,2016-08-08,0,2016


In [69]:
df_total[["garagetotalsqft","garagecarcnt"]].head()

,garagetotalsqft,garagecarcnt
0,0.0,2.0
1,0.0,1.0
2,441.0,2.0
3,460.0,2.0
4,665.0,2.0


In [70]:
df_total['garagecarcnt'].fillna(0, inplace=True)

In [72]:
value=df_total['garagetotalsqft'].median()
value

434.0

In [73]:
df_total['garagetotalsqft']=df_total['garagetotalsqft'].replace(float(0),np.nan)

In [74]:
m1=(df_total['garagecarcnt']>0.0)
m2=(df_total['garagecarcnt']==0.0)

df_total.loc[m1,'garagetotalsqft']=df_total.loc[m1,'garagetotalsqft'].fillna(value)
df_total.loc[m2,'garagetotalsqft']=df_total.loc[m2,'garagetotalsqft'].fillna(0.0)


In [75]:
df_total[["garagetotalsqft","garagecarcnt"]]

,garagetotalsqft,garagecarcnt
0,434.0,2.0
1,434.0,1.0
2,441.0,2.0
3,460.0,2.0
4,665.0,2.0
5,473.0,2.0
6,467.0,2.0
7,440.0,2.0
8,494.0,2.0
9,253.0,1.0


# Location Exploration

## Map The Log Errors

In [ ]:
sns.set()
plt.figure(figsize=(12,12))
sns.jointplot(x=df_total.latitude.values, y=df_total.longitude.values, size=10)
plt.ylabel('Longitude', fontsize=12)
plt.xlabel('Latitude', fontsize=12)
plt.show()

## Location Features

The location features are:
* latitude
* longitude
* regionidzip
* regionidcity
* regionidcounty
* regionidneighborhood
* fips
* censustractandblock
* rawcensustractandblock

Let's start with FIPS code, a federal code system for counties:

In [ ]:
df_total['fips'].value_counts()

To look up FIPS codes: https://www.census.gov/geo/reference/codes/cou.html

Counties in this set:
* 6037: LA County 
* 6059: Orange County 
* 6111: Ventura County

These should map 1:1 to regionidcounty values

In [ ]:
pd.crosstab(df_total['fips'],df_total['regionidcounty'])

Next, we'll look at the ZIP code data:

In [ ]:
df_total['regionidzip'].describe()

There appears to be an invalid US zip code for the max. Examine all impossible US zip codes:

In [ ]:
temp = df_total[df_total['regionidzip'] > 100000]
temp['regionidzip']

All of the entries have the same invalid zip. Look at the county the zip code is associated with.

In [ ]:
temp['fips'].value_counts()

All have the same county. Get all entries in that county:

In [ ]:
temp2 = df_total[df_total['fips'] == 6037]
temp2.groupby('regionidzip').count()

The zip code is most likely a military zip code. Let's look at some other features of the set:

In [ ]:
temp2['regionidzip'].mode()

This is not a US zip code. In spot checking, some of the zip codes are from CA, some are from OR, and some don't exist. Look at the other region identifiers:

In [ ]:


# 
df_merged.groupby('regionidcounty').count()

df_merged.groupby('regionidcity').count()

df_merged.groupby('regionidneighborhood').count()



# FIPS and RegionIDCounty contain identical information. For feature selection we will use FIPS since it has real-world meaning.

pd.crosstab(df_merged['regionidneighborhood'],df_merged['fips'])

pd.crosstab(df_merged['regionidcity'],df_merged['fips'])

pd.crosstab(df_merged['regionidzip'],df_merged['fips'])


nbcorr = df_merged[df_merged['fips']==6111]


pd.crosstab(nbcorr['regionidneighborhood'],nbcorr['regionidzip'])


# Even though the zip codes are fake, they do correspong to specific collections of neighborhoods, and it thus seems likely that Zillow did a 1:1 substitution when randomizing them. Since neighborhoods are more granular, they will be more useful for analysis.

# #### Latitude and Longitude

df_merged['latitude'].describe()

df_merged['longitude'].describe()


# Data Distributions

## Frequencies

In [ ]:
plt.close('all')
fig,ax=plt.subplots(figsize=(7, 7))
ax.set(yscale="symlog")
g=sns.distplot(df_num['logerror'].values, bins=50, kde=False)
#g.fig.get_axes()[0].set_yscale('log')
plt.show()

## Correlations

## Log Error Over Time

Let's look at the distribution of log errors over time:

In [ ]:
means = df_total.groupby('transactiondate')['logerror'].mean()

plt.close('all')
plt.figure(figsize=(20,5))
plt.scatter(df_total['transactiondate'].tolist(), df_total['logerror'], s =10, c = 'blue')
plt.scatter(means.index, means, s =10, c = 'red')
plt.title('LogError Over Time')
plt.xlabel('Transaction Date')
plt.ylabel('Logerror')
plt.show()
plt.close()

In [ ]:
df_total.groupby('setyear')['logerror'].describe()

The log error distributions are roughly consistent over time, with the annual means within one standard deviation of each other and an expected decrease in quantity during the winter months since there are fewer properties sold at that time of year.

# Missing Data and Data Anomalies

## Percent Missing Data

In [ ]:
missing_percents16 = (len(df_merged16.index) - df_merged16.count())/len(df_merged16.index)
missing_percents17 = (len(df_merged17.index) - df_merged17.count())/len(df_merged17.index)

In [ ]:
missing_percents16.sort_values(inplace=True)
temp = pd.DataFrame(missing_percents17, columns=['2017'])
missing_combined = pd.DataFrame(missing_percents16, columns=['2016'])
missing_combined = missing_combined.join(temp)

In [ ]:
missing_combined.plot.barh(figsize=(20,40))
plt.yticks(size=20)
plt.show()

## Outliers

# What Features Both Over and Underestimate the Log Error?